In [4]:
import numpy as np
from datetime import datetime
import os
import itertools as it
from env import *
import pandas as pd
import random
from Beurele_agent import *
from MO_agent import *


In [5]:
e=tiger_POMDP_env(read_config=True,config_address='./tiger.json',parameters=None)
num_to_act=dict(zip(list(e.actions.values()),list(e.actions.keys())))


{'state_change_r2l': 0.1, 'state_change_l2r': 0.1, 'false_observation_get_l_while_r': 0.3, 'false_observation_get_r_while_l': 0.3, 'reward_listen': -0.5, 'reward_low_incorrect': -1, 'reward_high_incorrect': -2, 'reward_low_correct': 1, 'reward_high_correct': 2, 'discount_factor': 0.9, 'initial_wealth': 0}


### Check similarity of methods' actions

In [ ]:
E=[-1.5,-0.2,-0.1,0.1,0.2,1.5]
W=[-1,0,1,2]
inits=[[1,0],[0.9,0.1],[0.75,0.25],[0.5,0.5]]
initial_observation=0

loop_counts=0
for max_depth in range(1,5):
    corr_count=0
    all_count=0
    loop_counts+=1
    print(loop_counts)
    for e1 in E:
        for e2 in E:
            exp_vorfaktoren=[e1,e2]
            for w1 in W:
                for w2 in W:
                    exp_weights=[w1,w2]
                    for init in inits:
                        ag2=Multi_Variate_agent(environment=e, planning_depth=max_depth, partitioning_chunk_number=10,agent_mode='cheating')
                        ag2.reset()
                        x_map2,M2,F2,G2,X2,value_function2,all_theta2=ag2.pre_planning(exp_vorfaktoren=exp_vorfaktoren,exp_weights=exp_weights, initial_theta=init,initial_observation=initial_observation)
                        v2,a2,q2,vf2=ag2.value_iteration()
                        
                        
                        ag=Bauerle_Rieder_agent(environment=e, num_of_Mu_chunks=2,max_iterations=max_depth)
                        b_S,b_r,b_map,b_q_func,b_value_function,b_action_func=ag.continious_optimized_planning(initial_mu_state=init,initial_wealth=0,exp_weights=exp_weights,exp_vorfaktoren=exp_vorfaktoren)
                        
                        #m_r,m_map, m_q_func,m_value_function,m_action_func=MO_cheating(env_dynamics=e,planning_depth=max_depth,exp_vorfaktoren=exp_vorfaktoren,exp_weights=exp_weights ,initial_theta=init,initial_observation=0,rounding_prec_coeff=10000)
                        #b_S,b_r,b_map,b_q_func,b_value_function,b_action_func=Beurele_cheating(env_dynamics=e,planning_depth=max_depth,exp_vorfaktoren=exp_vorfaktoren,exp_weights=exp_weights ,initial_mu_state=init,initial_wealth=0,comparison_precision=1.0e-5)
                        
                        m_init=init*len(init)
                        m_init.append(0)
                        
                        all_count+=1
                        if a2[0][tuple(m_init)]==b_action_func[0][0][tuple(init)]:
                            corr_count+=1
                        else:
                            print (q2[0][tuple(m_init)], b_q_func[0][0][tuple(init)],'           ',a2[0][tuple(m_init)],b_action_func[0][0][tuple(init)],'    ||     exp:',exp_vorfaktoren,exp_weights,init)
    print('--------------------------')                        
    print(all_count-corr_count,all_count,corr_count/all_count)
    print('=====================================')

## Simulation

### Beurele

In [6]:
max_depth=2
planning_depth=max_depth
exp_vorfaktoren=[-1,0.1]
exp_weights=[-1,1]
initial_Mu=[0.5,0.5]
initial_observation=0
initial_wealth=0

In [7]:
###############    Environment
e.reset()
initial_observation=e.current_state
observation=initial_observation
state=e.current_state
##############       Agent
# Create the agent
ag=Bauerle_Rieder_agent(environment=e, num_of_Mu_chunks=2,max_iterations=max_depth)

# Run its value iteration
b_S,b_reachables,b_map,b_q_func,b_value_function,b_action_func=ag.continious_optimized_planning(initial_mu_state=initial_Mu,initial_wealth=0,exp_weights=exp_weights,exp_vorfaktoren=exp_vorfaktoren)

# reset the agent 
ag.ch_reset(initial_mu_state=initial_Mu,initial_observation=initial_observation,initial_wealth=initial_wealth)

#############         Simulation

for t in range(planning_depth):
    
    print('t=',t)
    print('------')
    print('state:',state,e.states[state])
    print('last_observation:',observation)
    print('-------------------------------')
    print('')
    print('current internal state:',ag.current_internal_belief)
    print('')
    
    # agent select the action
    action,value_of_action,_=ag.ch_do_action()
    
    #environment feedback
    t1,t2,state,reward,observation=e.step(num_to_act[action])
    
    # agent update
    new_x=ag.ch_update_agent(new_observation=observation) 
    
    print('             action:',num_to_act[action])  
    print ('             **' )
    print('reward:',reward,'new observation:',observation)
    print('')
    print('new internal state:', ag.current_internal_belief)
    print('')
    print('============================')

t= 0
------
state: 0 tiger_right
last_observation: 0
-------------------------------

current internal state: [0.5, 0.5]

             action: listen
             **
reward: -0.5 new observation: 0

new internal state: [0.0, 0.0, 0.7, 0.0, 0.0, 0.0, 0.0, 0.30000000000000004, 0.0, 0.0]

t= 1
------
state: 0 tiger_right
last_observation: 0
-------------------------------

current internal state: [0.0, 0.0, 0.7, 0.0, 0.0, 0.0, 0.0, 0.30000000000000004, 0.0, 0.0]

             action: open_left_low
             **
reward: 1.0 new observation: 1

new internal state: [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.15000000000000002, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.35, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.15000000000000002, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.35, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]



### Multi Variate (Multi Objective)

In [9]:
planning_depth=2
partitioning_chunk_number=1000
exp_vorfaktoren=[-1,0.1]
exp_weights=[-1,1]
initial_theta=[0.5,0.5]
modes=['discrete_optimized','naive','cheating']

In [11]:
ag=Multi_Variate_agent(environment=e, planning_depth=planning_depth, partitioning_chunk_number=partitioning_chunk_number,agent_mode='cheating')
ag.reset()
mo_x_map,mo_M,mo_F,mo_G,mo_reachables,mo_value_function_zero,mo_all_thetas=ag.pre_planning(exp_vorfaktoren=exp_vorfaktoren,exp_weights=exp_weights, initial_theta=initial_theta,initial_observation=initial_observation)
mo_value_func,mo_action_func,mo_q_func,vf2=ag.value_iteration()

In [12]:
###############    Environment
e.reset()
initial_observation=e.current_state
observation=initial_observation
state=e.current_state

##############     Agent (above)

##############       Simulation
for t in range(planning_depth):
    
    print('t=',t)
    print('------')
    print('state:',state,e.states[state])
    print('last_observation:',observation)
    print('-------------------------------')
    print('')
    print('current internal state:',ag.current_internal_state)
    print('')
    
    # agent select the action
    action,value_of_action=ag.do_action()
    
    #environment feedback
    t1,t2,state,reward,observation=e.step(num_to_act[action])
    
    # agent update
    new_x=ag.update_agent(new_observation=observation) 
    
    print('             action:',num_to_act[action])  
    print ('             **' )
    print('reward:',reward,'new observation:',observation)
    print('')
    print('new internal state:', ag.current_internal_state)
    print('')
    print('============================')

t= 0
------
state: 1 tiger_left
last_observation: 1
-------------------------------

current internal state: [0.5, 0.5, 0.5, 0.5, 0]

             action: listen
             **
reward: -0.5 new observation: 0

new internal state: [0.7, 0.3, 0.7, 0.3, 0]

t= 1
------
state: 1 tiger_left
last_observation: 0
-------------------------------

current internal state: [0.7, 0.3, 0.7, 0.3, 0]

             action: open_left_low
             **
reward: -1.0 new observation: 1

new internal state: [0.5, 0.5, 0.5, 0.5, 1]

